Task 1

Apply at least one model from another group to the dataset to classify the industry and compare the performances

Task 2

Create a model using the word vector representation, how do you think this model performs compare to the others. Create a embeddings representation of each industry using spacy and find the closest industry using doc_1.similarity(industry_1)

Final task

Create a ReadMe.md on the base of your project to summarize the work, the challenges you faced and the new thing you want to explore

In [1]:
import pandas as pd
import sys
import numpy as np
import os
import re
import random
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import spacy

In [2]:

clean_df = pd.read_csv("clean_set.csv")
corpus = clean_df['clean_description'].values.astype(str)



In [3]:
industry_df = pd.read_csv('industry_data.csv')
industry_names= industry_df['industry'].values
training_corpus = industry_df['clean_description'].values.astype(str)


In [4]:
# Defining stopwords
with open("stopwords.txt", "r") as f_in:
    stopwords = [i.strip().lower() for i in f_in.readlines()]

Vectorizer

In [ ]:
# Parameters that we can tune
NGRAM = (1, 1) # Add more features when context is needed
MIN_DF = 2 # Ignore terms that appear less than 
MAX_DF = 0.4 # Ignore terms appear more than 30%
MAX_FEATURES = 4000 # Define the lenght of the vocabulary

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
idf_vec = TfidfVectorizer(
    tokenizer=lambda s: s.split(),
    stop_words=stopwords,
    min_df=MIN_DF,
    max_df=MAX_DF,
    #max_features=MAX_FEATURES,
    use_idf=True,
    smooth_idf=True
)

In [ ]:
industry=idf_vec.fit(training_corpus)
industry_vectors=industry.transform(training_corpus)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


In [ ]:
employer_vectors=idf_vec.transform(corpus)

### Training dataset with K-Means

In [ ]:
from sklearn.cluster import KMeans
kmeans=KMeans(n_clusters=13).fit(industry_vectors)

In [ ]:
prediction=kmeans.predict(employer_vectors)

In [ ]:
len(prediction)

20000

In [ ]:
kmeans.cluster_centers_

array([[0.        , 0.008433  , 0.        , ..., 0.        , 0.        ,
        0.0231945 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.01149929,
        0.01149929],
       [0.        , 0.        , 0.02666259, ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
kmeans_prediction=[]

In [ ]:
for i in prediction:
    kmeans_prediction.append(industry_names[i])

In [ ]:
clean_df['KMeans']=kmeans_prediction

In [ ]:
clean_df.to_csv('clean_set.csv', index=False)

### Word Vector Representation

In [20]:
!python -m spacy download en_core_web_md

2021-11-05 17:53:11.021979: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-05 17:53:11.022036: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
     |████████████████████████████████| 45.4 MB 42.9 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [21]:
model = spacy.load('en_core_web_md', disable=['parser', 'ner'])

In [6]:
# Parameters that we can tune
NGRAM = (1, 1) # Add more features when context is needed
MIN_DF = 2 # Ignore terms that appear less than 
MAX_DF = 0.4 # Ignore terms appear more than 30%
#MAX_FEATURES = 4000 # Define the lenght of the vocabulary

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    tokenizer=lambda s: s.split(),
    stop_words=stopwords,
    min_df=MIN_DF,
    max_df=MAX_DF,
    #max_features=MAX_FEATURES,
    use_idf=True,
    smooth_idf=True
)

In [8]:
employer_vectors=vectorizer.fit_transform(corpus)

In [ ]:
#employer=vectorizer.transform(corpus)

In [10]:
import numpy as np
top_n = 20

words = np.array(vectorizer.get_feature_names())
res = []
for i in range(employer_vectors.shape[0]):
    # Will get the words that are in the TFIDF which have the higher score
    # We use -vectors because the order is ascending
    s = np.argsort(np.asarray(-employer_vectors[i, :].todense()).flatten())
    res.append(" ".join(words[s[:top_n]]))

In [11]:
len(res)

20000

In [12]:
industry_vectors=vectorizer.fit_transform(training_corpus)

In [14]:
top_n = 20

words = np.array(vectorizer.get_feature_names())
industry = []
for i in range(industry_vectors.shape[0]):
    # Will get the words that are in the TFIDF which have the higher score
    # We use -vectors because the order is ascending
    s = np.argsort(np.asarray(-industry_vectors[i, :].todense()).flatten())
    industry.append(" ".join(words[s[:top_n]]))

In [15]:
len(industry)

13

In [22]:
#embedding words of industry 
industry_model=[]
for i in industry:
        industry_model.append(model(i))

In [23]:
industry_prediction=[]
#employer_model=[]

for i in res:
    employer_model=model(i)

    similarity_model=[]
    #len(employer_model)
    #industry_model=[]
    for k in industry_model:
        #employer_model.append(model(i))
        
        similarity_model.append(employer_model.similarity(k))     
    similar=np.argmax(similarity_model)                                     #getting highest similarity
    industry_prediction.append(industry_names[similar])

In [24]:
clean_df["embedding_prediction"] = industry_prediction

In [25]:
clean_df.to_csv('clean_set.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fb3de8ef-fb47-4eee-bfce-0a5c05122f97' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>